# 2. Production and Attraction Constrained Models



##### dis Constrained

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('data.csv')
#data.replace([np.inf, -np.inf], np.nan, inplace=True)
#data.dropna(inplace=True)

In [3]:
#create the formula (the "-1" indicates no intercept in the regression model).
formula = 'flows ~ station_destination + log_jobs + log_distance-1'
#run a production constrained sim
prodSim = smf.glm(formula = formula, data=data, family=sm.families.Poisson()).fit()
#let's have a look at it's summary
print(prodSim.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  flows   No. Observations:                61413
Model:                            GLM   Df Residuals:                    61014
Model Family:                 Poisson   Df Model:                          398
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.7321e+06
Date:                Wed, 24 Apr 2024   Deviance:                   3.2919e+06
Time:                        17:17:21   Pearson chi2:                 7.04e+06
No. Iterations:                     8   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
                                                       coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------

## 2.1 Model Estimates

In [4]:
O_i = pd.DataFrame(data.groupby(["station_origin"])["flows"].agg(np.sum))
O_i.rename(columns={"flows":"O_i"}, inplace = True)
cdatasub = data.merge(O_i, on = "station_origin", how = "left" )

D_j = pd.DataFrame(cdatasub.groupby(["station_destination"])["flows"].agg(np.sum))
D_j.rename(columns={"flows":"D_j"}, inplace = True)
cdatasub = cdatasub.merge(D_j, on = "station_destination", how = "left" )

/var/folders/60/b7504ly50v1_fz0lr9fy5tt00000gn/T/ipykernel_12558/1695467687.py:1: FutureWarning: The provided callable <function sum at 0x105c74700> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  O_i = pd.DataFrame(data.groupby(["station_origin"])["flows"].agg(np.sum))
/var/folders/60/b7504ly50v1_fz0lr9fy5tt00000gn/T/ipykernel_12558/1695467687.py:5: FutureWarning: The provided callable <function sum at 0x105c74700> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  D_j = pd.DataFrame(cdatasub.groupby(["station_destination"])["flows"].agg(np.sum))


In [5]:
cdatasub.head()

,station_origin,station_destination,flows,population,jobs,distance,unconstrainedEst1,log_population,log_jobs,log_distance,unconstrainedEst2,fitted,O_i,D_j
0,Abbey Road,Bank and Monument,0,599.1,78549.1,8131.625097,10,6.395429,11.271479,9.003516,50,49.622673,599,78549
1,Abbey Road,Beckton,1,599.1,442.1,8510.221774,0,6.395429,6.091536,9.049023,1,0.937341,599,442
2,Abbey Road,Blackwall,3,599.1,665.1,3775.548872,0,6.395429,6.499937,8.236301,2,2.121621,599,665
3,Abbey Road,Canary Wharf,1,599.1,58772.1,5086.614220,18,6.395429,10.981423,8.534368,53,53.302060,599,58772
4,Abbey Road,Canning Town,37,599.1,15428.1,2229.023167,25,6.395429,9.643946,7.709319,32,32.210949,599,15428


In [6]:
#We can do this by pulling out the parameter values
coefs = pd.DataFrame(prodSim.params)
coefs.reset_index(inplace=True)
coefs.rename(columns = {0:"alpha_i", "index":"coef"}, inplace = True)
to_repl = ["(station_destination)", "\[", "\]"]
for x in to_repl:
    coefs["coef"] = coefs["coef"].str.replace(x, "",regex=True)
#then once you have done this you can join them back into the dataframes
cdatasub = cdatasub.merge(coefs, left_on="station_destination", right_on="coef", how = "left")
cdatasub.drop(columns = ["coef"], inplace = True)
#check this has worked
cdatasub.head()

,station_origin,station_destination,flows,population,jobs,distance,unconstrainedEst1,log_population,log_jobs,log_distance,unconstrainedEst2,fitted,O_i,D_j,alpha_i
0,Abbey Road,Bank and Monument,0,599.1,78549.1,8131.625097,10,6.395429,11.271479,9.003516,50,49.622673,599,78549,-2.002235
1,Abbey Road,Beckton,1,599.1,442.1,8510.221774,0,6.395429,6.091536,9.049023,1,0.937341,599,442,1.199371
2,Abbey Road,Blackwall,3,599.1,665.1,3775.548872,0,6.395429,6.499937,8.236301,2,2.121621,599,665,0.477029
3,Abbey Road,Canary Wharf,1,599.1,58772.1,5086.614220,18,6.395429,10.981423,8.534368,53,53.302060,599,58772,-1.778337
4,Abbey Road,Canning Town,37,599.1,15428.1,2229.023167,25,6.395429,9.643946,7.709319,32,32.210949,599,15428,-1.357444


In [8]:
alpha_i = prodSim.params[0:397]
gamma = prodSim.params[398]
beta = -prodSim.params[399]

/var/folders/60/b7504ly50v1_fz0lr9fy5tt00000gn/T/ipykernel_12558/3448810109.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  gamma = prodSim.params[398]
/var/folders/60/b7504ly50v1_fz0lr9fy5tt00000gn/T/ipykernel_12558/3448810109.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  beta = -prodSim.params[399]


In [19]:
alpha_i

station_destination[Abbey Road]         0.998570
station_destination[Acton Central]      0.939810
station_destination[Acton Town]        -0.524100
station_destination[Aldgate]           -1.076271
station_destination[Aldgate East]      -1.139835
                                          ...   
station_destination[Wood Lane]         -0.383958
station_destination[Wood Street]        1.495963
station_destination[Woodford]           0.153539
station_destination[Woodgrange Park]    1.826946
station_destination[Woodside Park]      0.283216
Length: 397, dtype: float64

In [20]:
gamma

1.2391251001066133

In [11]:
beta

0.7211814583150664

In [12]:
cdatasub["prodsimest1"] = np.exp(cdatasub["alpha_i"]+gamma*cdatasub["log_jobs"]
                                 - beta*cdatasub["log_distance"])
#or you could do it the easy way like we did last week with the fitted column (See previous practical)
cdatasub.head(10)

,station_origin,station_destination,flows,population,jobs,distance,unconstrainedEst1,log_population,log_jobs,log_distance,unconstrainedEst2,fitted,O_i,D_j,alpha_i,prodsimest1
0,Abbey Road,Bank and Monument,0,599.1,78549.1,8131.625097,10,6.395429,11.271479,9.003516,50,49.622673,599,78549,-2.002235,237.785599
1,Abbey Road,Beckton,1,599.1,442.1,8510.221774,0,6.395429,6.091536,9.049023,1,0.937341,599,442,1.199371,9.221692
2,Abbey Road,Blackwall,3,599.1,665.1,3775.548872,0,6.395429,6.499937,8.236301,2,2.121621,599,665,0.477029,13.348400
3,Abbey Road,Canary Wharf,1,599.1,58772.1,5086.614220,18,6.395429,10.981423,8.534368,53,53.302060,599,58772,-1.778337,291.252770
4,Abbey Road,Canning Town,37,599.1,15428.1,2229.023167,25,6.395429,9.643946,7.709319,32,32.210949,599,15428,-1.357444,153.360839
5,Abbey Road,Crossharbour,1,599.1,1208.1,6686.575560,0,6.395429,7.096804,8.807857,2,2.340318,599,1208,0.389024,16.958834
6,Abbey Road,Custom House,0,599.1,845.1,3824.955630,0,6.395429,6.739455,8.249302,3,2.525158,599,845,0.518451,18.545638
7,Abbey Road,Cutty Sark,2,599.1,1748.1,8503.998909,0,6.395429,7.466285,9.048292,3,2.668811,599,1748,0.468617,24.405732
8,Abbey Road,Cyprus,7,599.1,850.1,6532.199618,0,6.395429,6.745354,8.784499,2,1.817492,599,850,0.789577,16.654776
9,Abbey Road,Devons Road,1,599.1,611.1,3958.424171,0,6.395429,6.415261,8.283601,2,1.931506,599,611,0.683132,14.274330


### 2.2.1 The flow matrics

In [13]:
#first round the estimates
cdatasub["prodsimest1"] = round(cdatasub["prodsimest1"],0)
#now we can create a pivot tabel to turn the paired list into a matrix, and compute the margins as well
cdatasubmat3 = cdatasub.pivot_table(values ="prodsimest1", index="station_destination", columns = "station_origin",
                            aggfunc=np.sum, margins=True)
cdatasubmat3

/var/folders/60/b7504ly50v1_fz0lr9fy5tt00000gn/T/ipykernel_12558/733211337.py:4: FutureWarning: The provided callable <function sum at 0x105c74700> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  cdatasubmat3 = cdatasub.pivot_table(values ="prodsimest1", index="station_destination", columns = "station_origin",
/var/folders/60/b7504ly50v1_fz0lr9fy5tt00000gn/T/ipykernel_12558/733211337.py:4: FutureWarning: The provided callable <function sum at 0x105c74700> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  cdatasubmat3 = cdatasub.pivot_table(values ="prodsimest1", index="station_destination", columns = "station_origin",
/var/folders/60/b7504ly50v1_fz0lr9fy5tt00000gn/T/ipykernel_12558/733211337.py:4: FutureWarning: The provided callable <fun

station_origin,Abbey Road,Acton Central,Acton Town,Aldgate,Aldgate East,All Saints,Alperton,Amersham,Anerley,Angel,...,Wimbledon,Wimbledon Park,Wood Green,Wood Lane,Wood Street,Woodford,Woodgrange Park,Woodside Park,Woolwich Arsenal,All
station_destination,,,,,,,,,,,,,,,,,,,,,
Abbey Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,348.0
Acton Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,752.0
Acton Town,NaN,NaN,NaN,7.0,7.0,NaN,18.0,4.0,NaN,8.0,...,8.0,8.0,6.0,15.0,NaN,5.0,NaN,5.0,NaN,2195.0
Aldgate,NaN,NaN,20.0,NaN,119.0,NaN,NaN,10.0,NaN,62.0,...,18.0,NaN,25.0,25.0,NaN,21.0,NaN,20.0,NaN,7778.0
Aldgate East,NaN,NaN,19.0,115.0,NaN,NaN,16.0,9.0,NaN,57.0,...,18.0,19.0,24.0,24.0,NaN,22.0,NaN,19.0,NaN,7933.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woodford,NaN,NaN,2.0,4.0,4.0,NaN,NaN,NaN,NaN,3.0,...,2.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,709.0
Woodgrange Park,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,242.0
Woodside Park,NaN,NaN,3.0,4.0,4.0,NaN,NaN,NaN,NaN,5.0,...,3.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,746.0


In [14]:
import scipy.stats
from math import sqrt
def CalcRSquared(observed, estimated):
    """Calculate the r^2 from a series of observed and estimated target values
    inputs:
    Observed: Series of actual observed values
    estimated: Series of predicted values"""
    
    r, p = scipy.stats.pearsonr(observed, estimated)
    R2 = r **2
    
    return R2


    

In [17]:
def CalcRMSE(observed, estimated):
    """Calculate Root Mean Square Error between a series of observed and estimated values
    inputs:
    Observed: Series of actual observed values
    estimated: Series of predicted values"""
    
    res = (observed -estimated)**2
    RMSE = round(sqrt(res.mean()), 3)

In [15]:
CalcRSquared(cdatasub["flows"], cdatasub["prodsimest1"])

0.0921274301205096

In [21]:
CalcRMSE(cdatasub["flows"], cdatasub["prodsimest1"])